In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
#import fifa21 file from desktop
data = pd.read_csv("fifa21_raw_data.csv")

/var/folders/gb/w4d_j72s4vb5n9fdl5qcclbh0000gn/T/ipykernel_44585/1934752076.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("fifa21_raw_data.csv")


### Data Preprocessing

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   photoUrl          18979 non-null  object
 1   LongName          18979 non-null  object
 2   playerUrl         18979 non-null  object
 3   Nationality       18979 non-null  object
 4   Positions         18979 non-null  object
 5   Name              18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Team & Contract   18979 non-null  object
 10  ID                18979 non-null  int64 
 11  Height            18979 non-null  object
 12  Weight            18979 non-null  object
 13  foot              18979 non-null  object
 14  BOV               18979 non-null  int64 
 15  BP                18979 non-null  object
 16  Growth            18979 non-null  int64 
 17  Joined      

In [4]:
data.describe(include="object")

,photoUrl,LongName,playerUrl,Nationality,Positions,Name,Team & Contract,Height,Weight,foot,...,Loan Date End,Value,Wage,Release Clause,W/F,SM,A/W,D/W,IR,Hits
count,18979,18979,18979,18979,18979,18979,18979,18979,18979,18979,...,1013,18979,18979,18979,18979,18979,18979,18979,18979,18979
unique,18978,18851,18978,164,640,17919,9023,21,56,2,...,24,214,141,1216,5,5,3,3,5,408
top,https://cdn.sofifa.com/players/251/698/21_60.png,Liam Kelly,http://sofifa.com/player/251698/kevin-berlaso/...,England,CB,J. Rodríguez,\n India\nFree\n\n,"6'0""",154lbs,Right,...,"Jun 30, 2021",€1.1M,€2K,€0,3 ★,2★,Medium,Medium,1 ★,\n1
freq,2,3,2,1704,2441,13,29,2859,1496,14445,...,770,467,2997,1262,11695,9142,12700,13956,17628,3216


In [5]:
#convert height to cm then change dtype to float
def convert_ht(ht):
    ht_ = ht.split("'") 
    ft = float(ht_[0])
    in_ = float(ht_[1].replace("\"",""))
    cm = ((12*ft) + in_ )*2.54
    return cm

data["Height"] = data["Height"].apply(lambda x: convert_ht(x))

In [6]:
#convert weight to kg then change dtype to float
def convert_wt(wt):
    wt_ = wt.split("lbs") 
    lb = float(wt_[0])
    kg = round(lb*0.45359237,1)
    return kg

data["Weight"] = data["Weight"].apply(lambda x: convert_wt(x))

In [7]:
data.Joined.value_counts()

Jul 1, 2019     1344
Jul 1, 2018      865
Jan 1, 2019      682
Jul 1, 2017      520
Jul 1, 2020      435
                ... 
May 17, 2018       1
Feb 25, 2015       1
Jun 1, 2005        1
Sep 10, 2016       1
Feb 28, 2015       1
Name: Joined, Length: 1869, dtype: int64

In [8]:
month_dict = {
    'jan' : 1,
    'feb' : 2,
    'mar' : 3,
    'apr' : 4,
    'may' : 5,
    'jun' : 6,
    'jul' : 7,
    'aug' : 8,
    'sep' : 9,
    'oct' : 10,
    'nov' : 11,
    'dec' : 12,
}

#convert column joined date fro easy splitting
def convertDay(date: str):
    month, day, year = re.findall(pattern=r"\w+", string=date) #select each individual attribute 
    month = month_dict.get(month.lower()) #using the month dictionary asign the appropriate value to each month
    if month != None and month/10 < 1: month = f"0{month}" #if a month is <10 add a 0 in front of it
    if day != None and len(day) < 2: day = f"0{day}" #if a day is < 10 add a 0 in front of it
    return f"{year}-{month}-{day}" #return a new date in format "2019-01-07" 

data.Joined = data.Joined.apply(convertDay)

In [9]:
blah = data["Joined"][0].split("-")
blah

['2004', '07', '01']

In [10]:
data[["year", "month", "day"]] = data["Joined"].str.split("-", expand = True)

In [11]:
data.Value

0        €67.5M
1          €46M
2          €75M
3          €87M
4          €90M
          ...  
18974      €35K
18975      €60K
18976      €40K
18977      €60K
18978      €60K
Name: Value, Length: 18979, dtype: object

In [12]:
def convertMoney(money: str):
    value = pd.to_numeric(re.findall(pattern=r"\d+\.?\d+|\d", string=money))[0]
    if value == 0:
        value = 0
    if "M" in money:
        value *= 1000000
    elif "K" in money:
        value *= 1000
    return str(int(value))

In [13]:
data.Value = pd.to_numeric(data.Value.apply(convertMoney))

In [14]:
data.Value

0        67500000
1        46000000
2        75000000
3        87000000
4        90000000
           ...   
18974       35000
18975       60000
18976       40000
18977       60000
18978       60000
Name: Value, Length: 18979, dtype: int64

In [15]:
data.Wage = pd.to_numeric(data.Wage.apply(convertMoney))
data['Release Clause'] = pd.to_numeric(data['Release Clause'].apply(convertMoney))

In [16]:
data.Wage

0        560000
1        220000
2        125000
3        370000
4        270000
          ...  
18974      1000
18975       500
18976      1000
18977       500
18978       500
Name: Wage, Length: 18979, dtype: int64

In [17]:
data.Hits

0        \n372
1        \n344
2         \n86
3        \n163
4        \n273
         ...  
18974        2
18975        3
18976        3
18977        5
18978        2
Name: Hits, Length: 18979, dtype: object

In [18]:
data['Hits'] = data['Hits'].astype(str)

In [19]:
data.Hits

0        \n372
1        \n344
2         \n86
3        \n163
4        \n273
         ...  
18974        2
18975        3
18976        3
18977        5
18978        2
Name: Hits, Length: 18979, dtype: object

In [20]:
data.Hits[0].replace("\n","")

'372'

In [21]:
data['Hits'] = data['Hits'].apply(lambda x: re.sub(pattern="\n", repl="", string=x))

In [22]:
data.Hits

0        372
1        344
2         86
3        163
4        273
        ... 
18974      2
18975      3
18976      3
18977      5
18978      2
Name: Hits, Length: 18979, dtype: object

In [23]:
data["Team & Contract"]

0                    \n\n\n\nFC Barcelona\n2004 ~ 2021\n\n
1                        \n\n\n\nJuventus\n2018 ~ 2022\n\n
2                 \n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n
3                 \n\n\n\nManchester City\n2015 ~ 2023\n\n
4             \n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n
                               ...                        
18974    \n\n\n\nChongqing Dangdai Lifan FC SWM Team\n2...
18975             \n\n\n\nOldham Athletic\n2020 ~ 2021\n\n
18976                  \n\n\n\nWuhan Zall\n2018 ~ 2022\n\n
18977             \n\n\n\nOldham Athletic\n2020 ~ 2021\n\n
18978                \n\n\n\nClub Bolívar\n2020 ~ 2024\n\n
Name: Team & Contract, Length: 18979, dtype: object

In [25]:
data["Team & Contract"] = data["Team & Contract"].str.replace('\n\n\n\n', '')

In [26]:
data["Team & Contract"]

0                            FC Barcelona\n2004 ~ 2021\n\n
1                                Juventus\n2018 ~ 2022\n\n
2                         Atlético Madrid\n2014 ~ 2023\n\n
3                         Manchester City\n2015 ~ 2023\n\n
4                     Paris Saint-Germain\n2017 ~ 2022\n\n
                               ...                        
18974    Chongqing Dangdai Lifan FC SWM Team\n2020 ~ 20...
18975                     Oldham Athletic\n2020 ~ 2021\n\n
18976                          Wuhan Zall\n2018 ~ 2022\n\n
18977                     Oldham Athletic\n2020 ~ 2021\n\n
18978                        Club Bolívar\n2020 ~ 2024\n\n
Name: Team & Contract, Length: 18979, dtype: object

In [27]:
data["Team & Contract"] = data["Team & Contract"].str.replace('\n\n', '')

In [28]:
data["Team & Contract"]

0                               FC Barcelona\n2004 ~ 2021
1                                   Juventus\n2018 ~ 2022
2                            Atlético Madrid\n2014 ~ 2023
3                            Manchester City\n2015 ~ 2023
4                        Paris Saint-Germain\n2017 ~ 2022
                               ...                       
18974    Chongqing Dangdai Lifan FC SWM Team\n2020 ~ 2020
18975                        Oldham Athletic\n2020 ~ 2021
18976                             Wuhan Zall\n2018 ~ 2022
18977                        Oldham Athletic\n2020 ~ 2021
18978                           Club Bolívar\n2020 ~ 2024
Name: Team & Contract, Length: 18979, dtype: object

In [29]:
team, contract = data["Team & Contract"][0].split("\n")

In [31]:
contract

'2004 ~ 2021'

In [35]:
data[["Team", "Contract"]] = data["Team & Contract"].str.split("\n", expand = True, n=1)

In [42]:
data.Team

0                               FC Barcelona
1                                   Juventus
2                            Atlético Madrid
3                            Manchester City
4                        Paris Saint-Germain
                        ...                 
18974    Chongqing Dangdai Lifan FC SWM Team
18975                        Oldham Athletic
18976                             Wuhan Zall
18977                        Oldham Athletic
18978                           Club Bolívar
Name: Team, Length: 18979, dtype: object

In [54]:
def sortContract(contract: str):
    if "\n" in contract:
        contract = contract.split("\n")[1]
    return contract

In [55]:
data.Contract = data.Contract.apply(sortContract)

In [57]:
data.Contract

0        2004 ~ 2021
1        2018 ~ 2022
2        2014 ~ 2023
3        2015 ~ 2023
4        2017 ~ 2022
            ...     
18974    2020 ~ 2020
18975    2020 ~ 2021
18976    2018 ~ 2022
18977    2020 ~ 2021
18978    2020 ~ 2024
Name: Contract, Length: 18979, dtype: object

In [56]:
data.Contract[data.Contract.str.contains("\n")].value_counts()

Series([], Name: Contract, dtype: int64)

In [50]:
string = 'blah-blah'
test = string.split('-')[1]
test

'blah'

In [ ]:
m = 4 
data[["Team", "Contract"]] = data["Team & Contract"].str.split("-", expand = True)

In [23]:
data["Team & Contract"] = data["Team & Contract"].apply(lambda x: re.sub(pattern="\n", repl="", string=x))

In [24]:
data["Team & Contract"]

0                               FC Barcelona2004 ~ 2021
1                                   Juventus2018 ~ 2022
2                            Atlético Madrid2014 ~ 2023
3                            Manchester City2015 ~ 2023
4                        Paris Saint-Germain2017 ~ 2022
                              ...                      
18974    Chongqing Dangdai Lifan FC SWM Team2020 ~ 2020
18975                        Oldham Athletic2020 ~ 2021
18976                             Wuhan Zall2018 ~ 2022
18977                        Oldham Athletic2020 ~ 2021
18978                           Club Bolívar2020 ~ 2024
Name: Team & Contract, Length: 18979, dtype: object